In [ ]:
import math
import time, json
from typing import Literal, Callable
import random
from random import randint

from drivingppo.common import set_seed, SPD_SCFAC, WORLD_DT, ACTION_REPEAT, LOOKAHEAD_POINTS
from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg
from drivingppo.simsim import WorldViewer

from world_samples import NEAR, gen_0, gen_1, gen_2, gen_2l, generate_random_world_plain, gen_obs

import numpy as np

W_CONFIG = {
    'map_border': False,
    'near': NEAR,
    'far': 999.9,
}


class PurePursuitController:
    def __init__(self, target_speed=SPD_SCFAC, min_ld=2.0, ld_k=1.0):
        """
        :param target_speed: 목표 주행 속도 (m/s)
        :param min_ld: 최소 전방 주시 거리 (m)
        :param ld_k: 속도 비례 전방 주시 계수 (Ld = min_ld + ld_k * speed)
        """
        self.target_speed = target_speed
        self.min_ld = min_ld
        self.ld_k = ld_k
        self.kp_steer = 1.5  # 조향 비례 제어 게인

    def get_action(self, world:World) -> tuple[float, float, bool]:
        """
        World 객체를 받아 강화학습 모델과 동일한 형태의 (ws, ad) 액션 반환
        """
        car = world.player
        waypoints = world.waypoints
        idx = world.waypoint_idx

        # 이미 도착한 경우 정지 명령
        if idx >= len(waypoints):
            return 0.0, 0.0, True

        ld = self.min_ld + self.ld_k * car.speed

        lookahead_point = waypoints[idx]
        cx, cz = car.x, car.z
        found = False

        # 현재 위치부터 남은 웨이포인트들을 선분으로 이어 교차점 검사
        pts = [(cx, cz)] + waypoints[idx:]

        for i in range(len(pts) - 1):
            p1x, p1z = pts[i]
            p2x, p2z = pts[i+1]

            # 선분(p1-p2)과 원(중심 cx,cz, 반지름 ld)의 수학적 교차점 계산
            dx, dz = p2x - p1x, p2z - p1z
            fx, fz = p1x - cx, p1z - cz

            a = dx*dx + dz*dz
            b = 2 * (fx*dx + fz*dz)
            c = (fx*fx + fz*fz) - ld*ld

            discriminant = b*b - 4*a*c
            if discriminant >= 0 and a > 0.00001:
                discriminant = math.sqrt(discriminant)
                t1 = (-b - discriminant) / (2*a)
                t2 = (-b + discriminant) / (2*a)

                # 진행 방향에 있는 선분 위의 교차점 (0 <= t <= 1) 선택
                # t2가 더 앞서 있는 점이므로 우선 채택
                if 0 <= t2 <= 1:
                    lookahead_point = (p1x + t2*dx, p1z + t2*dz)
                    found = True
                    break
                elif 0 <= t1 <= 1:
                    lookahead_point = (p1x + t1*dx, p1z + t1*dz)
                    found = True
                    break

        # 교차점을 못 찾았다면 (목표가 너무 가깝거나 경로 끝인 경우) 다음 웨이포인트를 쳐다봄
        if not found:
            lookahead_point = waypoints[min(idx + 1, len(waypoints)-1)]

        # 조향 제어 (ad: -1.0 ~ 1.0)
        tx, tz = lookahead_point
        abs_angle = angle_of(cx, cz, tx, tz)

        rel_angle = abs_angle - car.angle_x
        rel_angle = (rel_angle + math.pi) % (2*math.pi) - math.pi  # -pi ~ pi 정규화

        ad = self.kp_steer * rel_angle
        ad = max(min(ad, 1.0), -1.0)  # RL 액션 범위로 클리핑

        # 종방향 제어 (ws: -1.0 ~ 1.0)
        # 곡선 구간(조향각이 클 때)에서는 속도를 줄이는 간단한 로직 추가
        current_target_speed = self.target_speed * (1.0 - abs(ad) * 0.4)

        speed_error = current_target_speed - car.speed
        ws = speed_error * 0.5  # 가속도 비례 게인
        ws = max(min(ws, 1.0), -1.0)

        return ws, ad, False


def testew():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 50),
            (20, 50),
            (50, 70),
            (60, 60),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w

def gen_0():  return generate_random_world_plain(map_h= 50, map_w= 50, num=1,                min_dist=6,  max_dist=10, ang_init='half', ang_lim=0,      spd_init=0)
def gen_1():  return generate_random_world_plain(map_h=150, map_w=150, num=3,                min_dist=6,  max_dist=20, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')
def gen_2():  return generate_random_world_plain(map_h=150, map_w=150, num=LOOKAHEAD_POINTS, min_dist=6,  max_dist=30, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')
def gen_2l(): return generate_random_world_plain(map_h=150, map_w=150, num=20,               min_dist=6,  max_dist=30, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')

def run(
        world_generator:Callable[[], World],
        con:PurePursuitController,
        time_spd=2.0,
        time_step=WORLD_DT,
        step_per_control=ACTION_REPEAT,
        auto_close_at_end=True,
    ):

    world = world_generator()
    viewer = WorldViewer(world, auto_update=False)

    while not viewer.closed:

        if not world.arrived:
            ws, ad, _ = con.get_action(world)  # 컨트롤러가 조작 산출
            # ws, ad = 0.0, 0.0
            for _ in range(step_per_control):
                world.set_action(ws, ad)  # 행동 실행
                world.step(time_step)

        viewer.update()  # 시각화 호출
        time.sleep(time_step / 1000.0 / time_spd)  # 시각화 프레임을 위해 딜레이 추가


con = PurePursuitController()
set_seed(1)
run(testew, con)
